In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
from connect_impala import connect_impala
from impala.util import as_pandas
cur = connect_impala()

In [3]:
from  datetime import datetime,timedelta,date
today = datetime.strptime(
datetime.today().strftime('%Y-%m-%d'),
        '%Y-%m-%d')
yesterday = today+timedelta(days=1)
data_date=datetime.strptime('2017-11-12','%Y-%m-%d')
data_date1=datetime.strptime('2017-11-28','%Y-%m-%d')
date_start = datetime.strptime('2017-08-01','%Y-%m-%d')
date_end = datetime.strptime('2018-03-15','%Y-%m-%d')

In [31]:
sql_drop='drop table zybiro.beryl_sku_predict;'
cur.execute(sql_drop)

In [4]:
create_table = '''
      create table zybiro.beryl_sku_predict_temp0111
    (goods_id int,
    sku_id bigint,
     goods_number double)
    PARTITIONED BY (
      data_date STRING
      )
    '''
try:
    cur.execute(create_table)
except:
    print('已经建表')

In [5]:
days_len =(data_date-date_start).days
print(days_len)
print(date_start+timedelta(days=days_len))
for i in range(days_len):
    step_date  = date_start+timedelta(days=i)
    str_step = step_date.strftime('%Y-%m-%d') 
    print('='*5+str_step+'='*5)
    sql = '''
    insert overwrite table zybiro.beryl_sku_predict_temp0111  partition (data_date='{0}')
    select
    a.goods_id,
    a.sku_id,
    nvl(c.goods_number,0) goods_number
    from
 zybiro.beryl_chunjiebeihuo_goods_sku_info_temp0111confirmed a 
   left join
    (select
          to_date(
                  case when b.pay_id=41 
                  then b.pay_time 
                  else b.result_pay_time end) data_date,
         a.goods_id,
         a.sku_id,
         sum(a.goods_number)  goods_number ,
         sum(a.goods_revenue) goods_amount
         from 
         zydb.dw_order_goods_fact  a
         inner join zydb.dw_order_fact b 
             on a.order_id=b.order_id 
         where b.pay_status in (1,3)
         and  to_date(case when b.pay_id=41 then b.pay_time else b.result_pay_time end ) ="{0}" 
         group by  to_date(case when b.pay_id=41 then b.pay_time else b.result_pay_time end ) ,
         a.goods_id,
         a.sku_id
    ) c on a.goods_id=c.goods_id and  a.sku_id=c.sku_id 
    where a.indepot_num>0
    '''.format(str_step)
    cur.execute(sql)

103
2017-11-12 00:00:00
=====2017-08-01=====
=====2017-08-02=====
=====2017-08-03=====
=====2017-08-04=====
=====2017-08-05=====
=====2017-08-06=====
=====2017-08-07=====
=====2017-08-08=====
=====2017-08-09=====
=====2017-08-10=====
=====2017-08-11=====
=====2017-08-12=====
=====2017-08-13=====
=====2017-08-14=====
=====2017-08-15=====
=====2017-08-16=====
=====2017-08-17=====
=====2017-08-18=====
=====2017-08-19=====
=====2017-08-20=====
=====2017-08-21=====
=====2017-08-22=====
=====2017-08-23=====
=====2017-08-24=====
=====2017-08-25=====
=====2017-08-26=====
=====2017-08-27=====
=====2017-08-28=====
=====2017-08-29=====
=====2017-08-30=====
=====2017-08-31=====
=====2017-09-01=====
=====2017-09-02=====
=====2017-09-03=====
=====2017-09-04=====
=====2017-09-05=====
=====2017-09-06=====
=====2017-09-07=====
=====2017-09-08=====
=====2017-09-09=====
=====2017-09-10=====
=====2017-09-11=====
=====2017-09-12=====
=====2017-09-13=====
=====2017-09-14=====
=====2017-09-15=====
=====2017-

In [6]:
days_len2 = (data_date1-data_date).days+1
for i in range(days_len2):
    step_date  = data_date+timedelta(days=i)
    str_step = step_date.strftime('%Y-%m-%d')
    print('='*5+str_step+'='*5)
    sql = '''
     insert overwrite table zybiro.beryl_sku_predict_temp0111  partition (data_date='{0}')
    select 
    a.goods_id,
    a.sku_id,
    nvl((a.avg_num_90*1+a.avg_num_60*2+a.avg_num_30*3+a.avg_num_15*4+a.avg_num_7*5)/15,0) goods_number
    from
    (select
    aa.goods_id,
    aa.sku_id,
    nvl(aa.avg_num_90,0) avg_num_90,
    nvl(bb.avg_num_60,0) avg_num_60,
    nvl(cc.avg_num_30,0) avg_num_30,
    nvl(dd.avg_num_15,0) avg_num_15,
    nvl(ee.avg_num_7,0) avg_num_7
    from
    (select
            b.goods_id,
            b.sku_id,
            sum(a.goods_number*a.rank)/sum(a.rank)  avg_num_90
            from 
            (select 
            goods_id,
            sku_id,
            goods_number,
            ROW_NUMBER() OVER(partition by sku_id order by data_date asc)  rank
            from 
             zybiro.beryl_sku_predict_temp0111
            where to_date(data_date)<"{0}" 
            and  to_date(data_date)>=date_sub("{0}" ,90) ) a 
            right join  zybiro.beryl_chunjiebeihuo_goods_sku_info_temp0111confirmed b 
            on a.goods_id=b.goods_id and a.sku_id=b.sku_id
            where b.indepot_num>0
            group by b.goods_id ,b.sku_id) aa
       left join 
      (select
            b.goods_id,
            b.sku_id,
            sum(a.goods_number*a.rank)/sum(a.rank)  avg_num_60
            from 
            (select 
            goods_id,
            sku_id,
            goods_number,
            ROW_NUMBER() OVER(partition by sku_id order by data_date asc)  rank
            from 
             zybiro.beryl_sku_predict_temp0111
            where to_date(data_date)<"{0}" 
            and  to_date(data_date)>=date_sub("{0}" ,60) ) a 
            right join  zybiro.beryl_chunjiebeihuo_goods_sku_info_temp0111confirmed b 
            on a.goods_id=b.goods_id and a.sku_id=b.sku_id
            where b.indepot_num>0
            group by b.goods_id,b.sku_id ) bb
            on aa.goods_id=bb.goods_id and aa.sku_id=bb.sku_id
        left join
       (select
            b.goods_id,
            b.sku_id,
            sum(a.goods_number*a.rank)/sum(a.rank)  avg_num_30
            from 
            (select 
            goods_id,
            sku_id,
            goods_number,
            ROW_NUMBER() OVER(partition by sku_id order by data_date asc)  rank
            from 
             zybiro.beryl_sku_predict_temp0111
            where to_date(data_date)<"{0}" 
            and  to_date(data_date)>=date_sub("{0}" ,30) ) a 
            right join zybiro.beryl_chunjiebeihuo_goods_sku_info_temp0111confirmed b 
            on a.goods_id=b.goods_id and a.sku_id=b.sku_id
            where b.indepot_num>0
            group by b.goods_id,b.sku_id ) cc
            on aa.goods_id=cc.goods_id and aa.sku_id=cc.sku_id
       left join
       (select
            b.goods_id,
            b.sku_id,
            sum(a.goods_number*a.rank)/sum(a.rank)  avg_num_15
            from 
            (select 
            goods_id,
            sku_id,
            goods_number,
            ROW_NUMBER() OVER(partition by sku_id order by data_date asc)  rank
            from 
             zybiro.beryl_sku_predict_temp0111
            where to_date(data_date)<"{0}" 
            and  to_date(data_date)>=date_sub("{0}" ,15) ) a 
            right join zybiro.beryl_chunjiebeihuo_goods_sku_info_temp0111confirmed b 
            on a.goods_id=b.goods_id and a.sku_id=b.sku_id
            where b.indepot_num>0
            group by b.goods_id,b.sku_id ) dd
            on aa.goods_id=dd.goods_id and aa.sku_id=dd.sku_id
       left join
       (select
            b.goods_id,
            b.sku_id,
            sum(a.goods_number*a.rank)/sum(a.rank)  avg_num_7
            from 
            (select 
            goods_id,
            sku_id,
            goods_number,
            ROW_NUMBER() OVER(partition by sku_id order by data_date asc)  rank
            from 
            zybiro.beryl_sku_predict_temp0111
            where to_date(data_date)<"{0}" 
            and  to_date(data_date)>=date_sub("{0}" ,7) ) a 
            right join  zybiro.beryl_chunjiebeihuo_goods_sku_info_temp0111confirmed b 
            on a.goods_id=b.goods_id and a.sku_id=b.sku_id
            where b.indepot_num>0
            group by b.goods_id,b.sku_id ) ee
            on aa.goods_id=ee.goods_id and aa.sku_id=ee.sku_id
    ) a 
    
        '''.format(str_step)
    cur.execute(sql) 

=====2017-11-12=====
=====2017-11-13=====
=====2017-11-14=====
=====2017-11-15=====
=====2017-11-16=====
=====2017-11-17=====
=====2017-11-18=====
=====2017-11-19=====
=====2017-11-20=====
=====2017-11-21=====
=====2017-11-22=====
=====2017-11-23=====
=====2017-11-24=====
=====2017-11-25=====
=====2017-11-26=====
=====2017-11-27=====
=====2017-11-28=====


In [4]:
days_len = (today-data_date1).days
for i in range(days_len):
    step_date  =  data_date1+timedelta(days=i)
    str_step = step_date.strftime('%Y-%m-%d')
    print('='*5+str_step+'='*5)
    sql = '''
       insert overwrite table zybiro.beryl_sku_predict_temp0111  partition (data_date='{0}')
    select
    a.goods_id,
    a.sku_id,
    nvl(c.goods_number,0) goods_number
    from
 zybiro.beryl_chunjiebeihuo_goods_sku_info_temp0111confirmed a 
   left join
    (select
          to_date(
                  case when b.pay_id=41 
                  then b.pay_time 
                  else b.result_pay_time end) data_date,
         a.goods_id,
         a.sku_id,
         sum(a.goods_number)  goods_number ,
         sum(a.goods_revenue) goods_amount
         from 
         zydb.dw_order_goods_fact  a
         inner join zydb.dw_order_fact b 
             on a.order_id=b.order_id 
         where b.pay_status in (1,3)
         and  to_date(case when b.pay_id=41 then b.pay_time else b.result_pay_time end ) ="{0}" 
         group by  to_date(case when b.pay_id=41 then b.pay_time else b.result_pay_time end ) ,
         a.goods_id,
         a.sku_id
    ) c on a.goods_id=c.goods_id and  a.sku_id=c.sku_id 
    where a.indepot_num>0
    '''.format(str_step)
    cur.execute(sql)

=====2017-11-28=====
=====2017-11-29=====
=====2017-11-30=====
=====2017-12-01=====
=====2017-12-02=====
=====2017-12-03=====
=====2017-12-04=====
=====2017-12-05=====
=====2017-12-06=====
=====2017-12-07=====
=====2017-12-08=====
=====2017-12-09=====
=====2017-12-10=====
=====2017-12-11=====
=====2017-12-12=====
=====2017-12-13=====
=====2017-12-14=====
=====2017-12-15=====
=====2017-12-16=====
=====2017-12-17=====
=====2017-12-18=====
=====2017-12-19=====
=====2017-12-20=====
=====2017-12-21=====
=====2017-12-22=====
=====2017-12-23=====
=====2017-12-24=====
=====2017-12-25=====
=====2017-12-26=====
=====2017-12-27=====
=====2017-12-28=====
=====2017-12-29=====
=====2017-12-30=====
=====2017-12-31=====
=====2018-01-01=====
=====2018-01-02=====
=====2018-01-03=====
=====2018-01-04=====
=====2018-01-05=====
=====2018-01-06=====
=====2018-01-07=====
=====2018-01-08=====
=====2018-01-09=====
=====2018-01-10=====
=====2018-01-11=====
=====2018-01-12=====
=====2018-01-13=====
=====2018-01-

In [5]:
days_len2 = (date_end-today).days+1
for i in range(days_len2):
    step_date  = today+timedelta(days=i)
    str_step = step_date.strftime('%Y-%m-%d')
    print('='*5+str_step+'='*5)
    sql = '''
     insert overwrite table zybiro.beryl_sku_predict_temp0111  partition (data_date='{0}')
    select 
    a.goods_id,
    a.sku_id,
    nvl((a.avg_num_90*1+a.avg_num_60*2+a.avg_num_30*3+a.avg_num_15*4+a.avg_num_7*5)/15,0) goods_number
    from
    (select
    aa.goods_id,
    aa.sku_id,
    nvl(aa.avg_num_90,0) avg_num_90,
    nvl(bb.avg_num_60,0) avg_num_60,
    nvl(cc.avg_num_30,0) avg_num_30,
    nvl(dd.avg_num_15,0) avg_num_15,
    nvl(ee.avg_num_7,0) avg_num_7
    from
    (select
            b.goods_id,
            b.sku_id,
            sum(a.goods_number*a.rank)/sum(a.rank)  avg_num_90
            from 
            (select 
            goods_id,
            sku_id,
            goods_number,
            ROW_NUMBER() OVER(partition by sku_id order by data_date asc)  rank
            from 
             zybiro.beryl_sku_predict_temp0111
            where to_date(data_date)<"{0}" 
            and  to_date(data_date)>=date_sub("{0}" ,90) ) a 
            right join  zybiro.beryl_chunjiebeihuo_goods_sku_info_temp0111confirmed b 
            on a.goods_id=b.goods_id and a.sku_id=b.sku_id
            where b.indepot_num>0
            group by b.goods_id ,b.sku_id) aa
       left join 
      (select
            b.goods_id,
            b.sku_id,
            sum(a.goods_number*a.rank)/sum(a.rank)  avg_num_60
            from 
            (select 
            goods_id,
            sku_id,
            goods_number,
            ROW_NUMBER() OVER(partition by sku_id order by data_date asc)  rank
            from 
             zybiro.beryl_sku_predict_temp0111
            where to_date(data_date)<"{0}" 
            and  to_date(data_date)>=date_sub("{0}" ,60) ) a 
            right join  zybiro.beryl_chunjiebeihuo_goods_sku_info_temp0111confirmed b 
            on a.goods_id=b.goods_id and a.sku_id=b.sku_id
            where b.indepot_num>0
            group by b.goods_id,b.sku_id ) bb
            on aa.goods_id=bb.goods_id and aa.sku_id=bb.sku_id
        left join
       (select
            b.goods_id,
            b.sku_id,
            sum(a.goods_number*a.rank)/sum(a.rank)  avg_num_30
            from 
            (select 
            goods_id,
            sku_id,
            goods_number,
            ROW_NUMBER() OVER(partition by sku_id order by data_date asc)  rank
            from 
             zybiro.beryl_sku_predict_temp0111
            where to_date(data_date)<"{0}" 
            and  to_date(data_date)>=date_sub("{0}" ,30) ) a 
            right join zybiro.beryl_chunjiebeihuo_goods_sku_info_temp0111confirmed b 
            on a.goods_id=b.goods_id and a.sku_id=b.sku_id
            where b.indepot_num>0
            group by b.goods_id,b.sku_id ) cc
            on aa.goods_id=cc.goods_id and aa.sku_id=cc.sku_id
       left join
       (select
            b.goods_id,
            b.sku_id,
            sum(a.goods_number*a.rank)/sum(a.rank)  avg_num_15
            from 
            (select 
            goods_id,
            sku_id,
            goods_number,
            ROW_NUMBER() OVER(partition by sku_id order by data_date asc)  rank
            from 
             zybiro.beryl_sku_predict_temp0111
            where to_date(data_date)<"{0}" 
            and  to_date(data_date)>=date_sub("{0}" ,15) ) a 
            right join zybiro.beryl_chunjiebeihuo_goods_sku_info_temp0111confirmed b 
            on a.goods_id=b.goods_id and a.sku_id=b.sku_id
            where b.indepot_num>0
            group by b.goods_id,b.sku_id ) dd
            on aa.goods_id=dd.goods_id and aa.sku_id=dd.sku_id
       left join
       (select
            b.goods_id,
            b.sku_id,
            sum(a.goods_number*a.rank)/sum(a.rank)  avg_num_7
            from 
            (select 
            goods_id,
            sku_id,
            goods_number,
            ROW_NUMBER() OVER(partition by sku_id order by data_date asc)  rank
            from 
            zybiro.beryl_sku_predict_temp0111
            where to_date(data_date)<"{0}" 
            and  to_date(data_date)>=date_sub("{0}" ,7) ) a 
            right join  zybiro.beryl_chunjiebeihuo_goods_sku_info_temp0111confirmed b 
            on a.goods_id=b.goods_id and a.sku_id=b.sku_id
            where b.indepot_num>0
            group by b.goods_id,b.sku_id ) ee
            on aa.goods_id=ee.goods_id and aa.sku_id=ee.sku_id
    ) a 
        '''.format(str_step)
    cur.execute(sql) 

=====2018-02-23=====
=====2018-02-24=====
=====2018-02-25=====
=====2018-02-26=====
=====2018-02-27=====
=====2018-02-28=====
=====2018-03-01=====
=====2018-03-02=====
=====2018-03-03=====
=====2018-03-04=====
=====2018-03-05=====
=====2018-03-06=====
=====2018-03-07=====
=====2018-03-08=====
=====2018-03-09=====
=====2018-03-10=====
=====2018-03-11=====
=====2018-03-12=====
=====2018-03-13=====
=====2018-03-14=====
=====2018-03-15=====
